## Import python libs and setup FLAGS

In [1]:
import cifar10
import os
import tensorflow as tf
from AlexNet import alexnet
import tf_general as tfg
import numpy as np

FLAGS = tf.flags.FLAGS
try:
    tf.app.flags.DEFINE_string('f', '', 'kernel')
    tf.flags.DEFINE_integer('epoch', 50000, 'epoch')
    tf.flags.DEFINE_integer('batch_size', 128, 'batch size')
    tf.flags.DEFINE_float('lr', 0.01, 'learning rate')
    tf.flags.DEFINE_integer('image_size', 225, 'image size')
    tf.flags.DEFINE_integer('test_size', 512, 'test size')
    #tf.flags.DEFINE_boolean('restore', False, 'restore from checkpoint and run test')
    print('parameters are defined.')
except:
    print('parameters have been defined.')

parameters are defined.


## Define the data source. 
The data path is 'cifar-10-batches'. Use distorted_input() to get training data and inputs() to get test data.

In [2]:
CONTINUE = 0
#第三次微调
cifar10_dir='cifar-10-batches' 
#获取数据增强后的训练集数据
train_image, train_label = cifar10.distorted_inputs(cifar10_dir,FLAGS.batch_size, FLAGS.image_size)
#获取数据裁剪后的测试数据
#test_image, test_label = cifar10.inputs(True, cifar10_dir, FLAGS.test_size, FLAGS.image_size)

Filling queue with 20000 CIFAR images before starting to train. This will take a few minutes.


## define input layer

In [3]:
with tf.name_scope('input'):
    #x = tf.placeholder(tf.float32, [None, 225,225,3], name='input_image')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    global_step = tf.placeholder(tf.int32, name='global_step')
    y = train_label
    #tf.placeholder(tf.int64, name='input_label')

## define alexnet object 

In [4]:
with tf.name_scope('prediction'):
    alex_net = alexnet(train_image, keep_prob)
    y_ = alex_net.prediction

norm_1:  (128, 225, 225, 3)
conv_1:  (128, 54, 54, 96)
lrn_1:  (128, 54, 54, 96)
pool_1:  (128, 26, 26, 96)
conv_2:  (128, 26, 26, 256)
lrn_2:  (128, 26, 26, 256)
pool_2:  (128, 12, 12, 256)
conv_3:  (128, 12, 12, 384)
conv_4:  (128, 12, 12, 256)
conv_5:  (128, 12, 12, 256)
pool_3:  (128, 5, 5, 256)
flat_1:  (128, 6400)
fc_1:  (128, 4096)
fc_2:  (128, 4096)
fc_3:  (128, 10)
drop_out_1:  (128, 10)


## Use cross entropy as loss

In [5]:
with tf.name_scope('cross_entropy'):
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y_,
                           labels=y, name="cross_entropy_per_example")
    cross_entropy = tf.reduce_mean(cross_entropy)
    print('calculate cross entropy')

calculate cross entropy


## Use optimizer to optimize parameters

In [6]:
with tf.name_scope('train_step'):
    train_step = tf.train.AdagradOptimizer(FLAGS.lr).minimize(cross_entropy)
    print('set train step')

set train step


## Calculate training accuracy

In [7]:
accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(y_, 1), y), tf.float32))
print('set accuracy')

set accuracy


## Set up model saving path

In [ ]:
ckpt_dir = 'ckpt/'
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)
saver = tf.train.Saver(max_to_keep = 1)
coord = tf.train.Coordinator()

## Start session to run the training

In [ ]:
import tf_general as tfg
import time

if __name__ == '__main__':
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        if CONTINUE != 0:
            model_file=tf.train.latest_checkpoint('ckpt/')
            saver.restore(sess, model_file)
        threads = tf.train.start_queue_runners(sess, coord)
        i = 0
        try:
            while not coord.should_stop() and i< FLAGS.epoch:
                i += 1 
                loss,_,accuracy_rate = sess.run([cross_entropy,train_step,accuracy], 
                                                       feed_dict={keep_prob: 0.5, global_step: i}) 
                log = []
                string = time.strftime("%Y-%m-%d %X",time.localtime())+ ' iter:' + str(i)+'  '+str(round(accuracy_rate*100,2))+ '%'
                print(string)
                log.append(string)
                if (i+1)%250 == 0 and i > 0:
                    string = time.strftime("%Y-%m-%d %X",time.localtime())+ ' save model at step '+ str(i) 
                    print(string)
                    log.append(string)
                    saver.save(sess,'ckpt/cifar10_'+str(CONTINUE)+'_'+str(i)+'.ckpt',global_step=i)
                tfg.saveEvalData('./alexnet_cifar_'+str(CONTINUE)+'.log',log)
        except tf.errors.OutOfRangeError:
            string = time.strftime("%Y-%m-%d %X",time.localtime())+ " done! now lets kill all the threads……"
            log = []
            log.append(string)
            tfg.saveEvalData('./alexnet_cifar_'+str(CONTINUE)+'.log',log)
        finally:
            string = time.strftime("%Y-%m-%d %X",time.localtime()) + " Finally completed!"
            log = []
            log.append(string)
            tfg.saveEvalData('./alexnet_cifar_'+str(CONTINUE)+'.log',log)
            coord.request_stop()

2019-05-24 08:01:15 iter:1  4.69%
2019-05-24 08:02:00 iter:2  14.06%
2019-05-24 08:02:44 iter:3  10.94%
2019-05-24 08:03:29 iter:4  14.06%
2019-05-24 08:04:14 iter:5  12.5%
2019-05-24 08:04:58 iter:6  7.03%
2019-05-24 08:05:43 iter:7  10.94%
2019-05-24 08:06:28 iter:8  6.25%
2019-05-24 08:07:13 iter:9  13.28%
2019-05-24 08:07:57 iter:10  11.72%
2019-05-24 08:08:42 iter:11  6.25%
2019-05-24 08:09:27 iter:12  10.16%
2019-05-24 08:10:11 iter:13  7.03%
2019-05-24 08:10:56 iter:14  13.28%
2019-05-24 08:11:41 iter:15  3.91%
2019-05-24 08:12:26 iter:16  10.16%
